In [6]:
from google.colab import drive # 구글 드라이브 연결
drive.mount('/content/drive')

Mounted at /content/drive


In [9]:
csv_file_path = "./drive/MyDrive/combined_dataset.csv"
stop_words_path = "./drive/MyDrive/stopword.txt"

In [15]:
import pandas as pd
data = pd.read_csv(csv_file_path, sep=",")
data.head()

,id,category,original_text,simple_text,source,collector
0,1,식사,"규칙적인 식사와 고른 영양섭취, 충분한 야채, 과일 및 단백질 섭취는 치료 전후 회...","밥은 매일 잘 먹고, 채소랑 과일이랑 고기도 골고루 먹으면 좋아요.",https://www.snuh.org/board/B016/view.do?bbs_no...,김하영
1,2,운동,퇴원 후 집안에서 생활하는 데는 지장이 없습니다.,병원에서 집에 온 뒤에도 집에서 잘 지낼 수 있어요.,https://www.snuh.org/board/B016/view.do?bbs_no...,김하영
2,3,운동,2주 정도는 배에 힘이 들어가는 무리한 운동을 삼가십시오.,배에 힘이 많이 들어가는 놀이는 2주 동안 하지 마세요.,https://www.snuh.org/board/B016/view.do?bbs_no...,김하영
3,4,목욕 및 샤워,실밥제거 후에는 다음 날부터 가벼운 샤워를 시작해도 됩니다.,꿰맨 실을 빼고 나면 다음 날 씻어도 괜찮아요.,https://www.snuh.org/board/B016/view.do?bbs_no...,김하영
4,5,목욕 및 샤워,상처부위의 때를 밀거나 문질러 닦는 것은 상처가 완전히 아문 다음 시작하십시오.,다 나을 때까지 아픈 데를 밀거나 세게 닦지 마세요.,https://www.snuh.org/board/B016/view.do?bbs_no...,김하영


In [16]:
with open(stop_words_path, "r", encoding="UTF-8") as inFile:
  lines = inFile.readlines()

stop_words = set([])
for line in lines:
  stop_words.add(line.replace("\n",""))

In [17]:
!pip install kiwipiepy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 35.5/35.5 MB 54.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 92.4 MB/s eta 0:00:00
  Created wheel for kiwipiepy_model: filename=kiwipiepy_model-0.21.0-py3-none-any.whl size=35593192 sha256=5532f962d96d5c47d880e9c1272b5e8b7a4dadcba254676bd7fc00cc85fe62b5
  Stored in directory: /root/.cache/pip/wheels/b0/16/3d/95053ab5298f0f0f22ffea6de0200b6f24bffb73cab4c1a828
Successfully built kiwipiepy_model


In [18]:
from kiwipiepy import Kiwi
from kiwipiepy.utils import Stopwords
import re
# Kiwi의 객체 생성
kiwi = Kiwi()
stopwords = Stopwords()

In [19]:
def preprocessing(kiwi, review):
  # 한글이 아닌 단어는 지우고 " "으로 대체
  review_text = re.sub(r'[^ㄱ-ㅎ가-힣]+', ' ', review)

  token_list = kiwi.tokenize(review_text, stopwords=stopwords) # 받침 분리 안함 + stopwords
  # 초성에 해당하지 않는 단어만 clean_review에 들어갈 수 있게 함
  clean_review = []
  for token in token_list:
    if not token[1] == 'ETM' and not token[1] == 'EF':
      # if token[1] == 'EF': # 됩니다. 하십시오. 등은 되다. 하오. 로 전처리 하기위함
      #   clean_review[-1] += token[0][-1]
      # else:
      #   clean_review.append(token[0])
      clean_review.append(token[0])
  # 단어들을 ' '기준으로 이어 붙혀서 하나의 string 타입의 변수를 생성함
  clean_review = ' '.join(clean_review)

  # 전처리 완료된 문장을 return
  return clean_review.strip()

In [20]:
df = data
df['preprocessing_text'] = df['original_text'].apply(lambda t : preprocessing(kiwi, t))

In [21]:
df.head()

,id,category,original_text,simple_text,source,collector,preprocessing_text
0,1,식사,"규칙적인 식사와 고른 영양섭취, 충분한 야채, 과일 및 단백질 섭취는 치료 전후 회...","밥은 매일 잘 먹고, 채소랑 과일이랑 고기도 골고루 먹으면 좋아요.",https://www.snuh.org/board/B016/view.do?bbs_no...,김하영,규칙 식사 고르 영양 섭취 충분 야채 과일 및 단백질 섭취 치료 전후 회복 도움
1,2,운동,퇴원 후 집안에서 생활하는 데는 지장이 없습니다.,병원에서 집에 온 뒤에도 집에서 잘 지낼 수 있어요.,https://www.snuh.org/board/B016/view.do?bbs_no...,김하영,퇴원 후 집안 생활 데 지장
2,3,운동,2주 정도는 배에 힘이 들어가는 무리한 운동을 삼가십시오.,배에 힘이 많이 들어가는 놀이는 2주 동안 하지 마세요.,https://www.snuh.org/board/B016/view.do?bbs_no...,김하영,주 정도 배 힘 들어가 무리 운동 삼가 시
3,4,목욕 및 샤워,실밥제거 후에는 다음 날부터 가벼운 샤워를 시작해도 됩니다.,꿰맨 실을 빼고 나면 다음 날 씻어도 괜찮아요.,https://www.snuh.org/board/B016/view.do?bbs_no...,김하영,실밥 제거 후 다음 날 가볍 샤워 시작 어도
4,5,목욕 및 샤워,상처부위의 때를 밀거나 문질러 닦는 것은 상처가 완전히 아문 다음 시작하십시오.,다 나을 때까지 아픈 데를 밀거나 세게 닦지 마세요.,https://www.snuh.org/board/B016/view.do?bbs_no...,김하영,상처 부위 밀 거나 문지르 닦 상처 완전히 아물 다음 시작 시


In [22]:
df.to_csv(path_or_buf="./drive/MyDrive/preprocessing_dataset.csv")